In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
datapath = "../data/data/v2.0/"
import csv
from collections import defaultdict
import seaborn as sn
from numpy.random import default_rng
from sklearn import tree
import decisiontree as dt

In [2]:
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [110]:
english_dataframe = pd.read_csv(datapath+"hipe2020/en/HIPE-2022-v2.0-hipe2020-dev-en.tsv", sep = '\t', header = 0, engine="python", comment='#', quoting=csv.QUOTE_NONE)

In [54]:
english_dataframe.loc[98]

TOKEN                       ar
NE-COARSE-LIT                O
NE-COARSE-METO               O
NE-FINE-LIT                  _
NE-FINE-METO                 _
NE-FINE-COMP                 _
NE-NESTED                    _
NEL-LIT                      _
NEL-METO                     _
MISC              NoSpaceAfter
SENTENCE_ID                  4
Name: 98, dtype: object

In [90]:
# combine_sign = "¬"
# del_list = []
# english_dataframe = english_dataframe.copy()
# for i in range(len(english_dataframe)):
#     if english_dataframe.loc[i]["MISC"] == "NoSpaceAfter" and english_dataframe.loc[i+1]["TOKEN"] == combine_sign and english_dataframe.loc[i+1]["MISC"] == "EndOfLine":
#         new_token = english_dataframe.loc[i]["TOKEN"] + english_dataframe.loc[i+2]["TOKEN"]
#         english_dataframe.at[i,"TOKEN"] = new_token
#         del_list += [i+1, i+2]

In [91]:
# english_dataframe.drop(index = del_list, inplace=True)
# english_dataframe.reset_index(drop=True, inplace=True)

In [86]:
test_dataframe.head(n=105)

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC,SENTENCE_ID
0,From,O,O,_,_,_,_,_,_,_,0
1,the,O,O,_,_,_,_,_,_,_,0
2,AMERICAN,B-prod,O,_,_,_,_,NIL,_,_,0
3,MERCURY,I-prod,O,_,_,_,_,NIL,_,NoSpaceAfter,0
4,.,O,O,_,_,_,_,_,_,EndOfLine|EndOfSentence,0
...,...,...,...,...,...,...,...,...,...,...,...
100,leading,O,O,_,_,_,_,_,_,_,4
101,principles,O,O,_,_,_,_,_,_,NoSpaceAfter,4
102,",",O,O,_,_,_,_,_,_,_,4
103,from,O,O,_,_,_,_,_,_,_,4


In [88]:
test_dataframe.loc[98]

TOKEN                arguments
NE-COARSE-LIT                O
NE-COARSE-METO               O
NE-FINE-LIT                  _
NE-FINE-METO                 _
NE-FINE-COMP                 _
NE-NESTED                    _
NEL-LIT                      _
NEL-METO                     _
MISC              NoSpaceAfter
SENTENCE_ID                  4
Name: 98, dtype: object

In [111]:
misc_list = english_dataframe["MISC"].to_list()

In [112]:
sentence_idx = []
idx = 0
for item in misc_list:
    sentence_idx.append(idx)
    if "EndOfSentence" in item:
        idx += 1
english_dataframe["SENTENCE_ID"] = sentence_idx

In [113]:
labelled_sentences = set()
for i, label in enumerate(english_dataframe["NE-COARSE-LIT"].to_list()):
    if label != "O":
        labelled_sentences.add(sentence_idx[i])

In [114]:
token_dict = english_dataframe["TOKEN"].str.lower().value_counts().to_dict()

In [115]:
NE_dict = english_dataframe[english_dataframe["NE-COARSE-LIT"] != "O"]["TOKEN"].value_counts().to_dict()

In [116]:
feature_list = []
total_word_count = len(english_dataframe)
total = 0
for key in token_dict:
    total += token_dict[key]
    feature_list.append(key)
    if total > total_word_count * 0.5:
        break
print(total)
print(feature_list)
print(len(feature_list))

14543
[',', 'the', '.', 'of', 'and', 'to', 'a', '¬', 'in', 'is', '-', 'that', 'for', 'he', 'be', 'as', 'was', 'it', 'at', 'on', 'by', 'with', 'this', 'will', 'have', 'from', '’', 'his', 'are', 'i', 'their', ';', 'which', 's', 'not', 'they', '*', 'has', '“', "'", 'you', 'an', 'or', 'one', 'all', '”', 'we', 'been', 'but', 'no', 'who', 'than', 'our', 'were', 'other', '—', 'county', 'any', 'them', 'made', 'when', 'man', 'more', 'new', 'there', 'out', 'can', 'j', 'may', 'day', 'would', 'its', ':', 'had', '(']
75


In [117]:
post_fix_dict_3 = defaultdict(lambda:0)
post_fix_dict_2 = defaultdict(lambda:0)
post_fix_3_list = []
post_fix_2_list = []
for key in token_dict:
    if key == "endofsentence":
        continue
    if len(key) <= 2:
        continue
    count = token_dict[key]
    
    if len(key) == 3:
        post_fix_2 = key[-2:]
        post_fix_dict_2[post_fix_2] += count
        if post_fix_dict_2[post_fix_2] >= 30 and post_fix_2 not in post_fix_2_list:
            post_fix_2_list.append(post_fix_2)
        continue

    post_fix_3 = key[-3:]
    post_fix_dict_3[post_fix_3] += count
    if post_fix_dict_3[post_fix_3] >= 30 and post_fix_3 not in post_fix_3_list:
            post_fix_3_list.append(post_fix_3)
    

In [118]:
label_list =  list(pd.unique(english_dataframe["NE-COARSE-LIT"]))

In [119]:
feature_matrix = []
label_matrix = []
pos_dict = defaultdict(lambda: [])
label_dict = defaultdict(lambda: [])
pos = 0
progress = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
prev_label = 0
extra_features = 6
check = True
for idx in english_dataframe.index:
    if idx/len(english_dataframe) > progress[0]:
        print(progress[0])
        progress.pop(0)
    pos += 1
    row = english_dataframe.iloc[idx]
    token = row["TOKEN"]
    if isinstance(token, float): #null in the tsv is taken as a nan value
        token = "null"
    prev_label = label
    label = int(row["NE-COARSE-LIT"] != "O")
    if token.lower() in feature_list:
        pos_dict[token.lower() ].append(pos)
    if "EndOfSentence" not in row["MISC"]:
        label_dict[token].append((pos,label,prev_label,0))
        continue
    label_dict[token].append((pos,label,prev_label,1))
    # loop of all tokens in a sentence
    for token in label_dict:
        # loop over all ocurances of a token
        for item in label_dict[token]:
            feature_vector = np.zeros(shape = (len(feature_list)+extra_features)) #differntiate between wheter a token occurs or wheter the current token is in the top 50%
            # label_vector = np.zeros(shape=(2))
            current_pos = item[0]
            
            # loop over all features present in a sentence
            for feature in pos_dict:
                closest_pos = pos_dict[feature][0] - current_pos # starting point
                
                # loop over all occurences of a feature present in a sentence
                for pos in pos_dict[feature]:
                    relative_pos = pos - current_pos
                    if abs(relative_pos) < abs(closest_pos):
                        closest_pos = relative_pos
                    else: break
                feature_vector[feature_list.index(feature)] = closest_pos
            feature_vector[-1] = item[0] # add location as feature
            feature_vector[-2] = item[3] # add wheter the item is the end of a sentence
            feature_vector[-3] = int(token[0].isupper()) #add wheter the first char is a capital letter
            feature_vector[-4] = int(any(char.isdigit() for char in token))
            
            if len(token) > 3:
                post_fix = token[-3:].lower()
                if post_fix in post_fix_3_list:
                    feature_vector[-5] = post_fix_3_list.index(post_fix) + 1
                    check = False
                # print(post_fix)
            if len(token) > 2:
                post_fix = token[-2:].lower()
                if post_fix in post_fix_2_list:
                    feature_vector[-6] = post_fix_2_list.index(post_fix) + 1
                # print(post_fix)
            # if not check:
            #     print(token)
            #     break
            feature_matrix.append(feature_vector)
            # label_vector[item[1]] = 1
            label_matrix.append(item[1])
    # break
    pos = 0
    pos_dict = defaultdict(lambda: [])
    label_dict = defaultdict(lambda: [])
    prev_label = 0
feature_matrix = np.array(feature_matrix)
label_matrix = np.array(label_matrix)


0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [120]:
total_sentences = len(set(sentence_idx))
split = 0.5
rng = default_rng()
training_sentence_ids = rng.choice(total_sentences, size=int(total_sentences*split), replace=False)
test_sentence_ids = [i for i in range(total_sentences) if i not in training_sentence_ids]

training_id_list = []
test_id_list = []
for i, item in enumerate(sentence_idx):
    if item in training_sentence_ids:
        training_id_list.append(i)
    else:
        test_id_list.append(i)

training_feature_matrix = np.array([feature_matrix[i] for i in training_id_list])
training_label_matrix = np.array([label_matrix[i] for i in training_id_list])
test_feature_matrix = np.array([feature_matrix[i] for i in test_id_list])
test_label_matrix = np.array([label_matrix[i] for i in test_id_list])

In [14]:
print(training_feature_matrix.shape)

(14663, 81)


In [121]:
total_features = training_feature_matrix.shape[1]
discrete_idx_list = [total_features-2, total_features-3, total_features-4, total_features-5, total_features-6]

In [16]:
print(discrete_idx_list)

[79, 78, 77, 76, 75]


In [123]:
test_classifier = dt.custom_decision_tree(training_feature_matrix, training_label_matrix, discrete_idx_list=discrete_idx_list, min_samples=35)
test_classifier.build_tree()

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [124]:
custom_tree = test_classifier.tree

In [23]:
custom_tree.print_node()

feature: 78
best guess: 0
entropy: 0.4381362130723046
split values [0.0, 1.0]
type: discrete
child value: 0.0    child branch feature: 76
child value: 1.0    child branch feature: 76


In [128]:
tp = 0
fp = 0
fn = 0
tn = 0
total = 0
prediction = custom_tree.predict_matrix(test_feature_matrix)
prev_label = 0
for i, pred in enumerate(prediction):
    correct_label = test_label_matrix[i]
    if correct_label == 0 and pred == 0:
        tn += 1
    elif correct_label == 0 and pred != 0:
        fn += 1
    elif correct_label == 1 and pred == 0:
        fn += 1
    else:
        tp += 1

accuracy = (tp + tn) / (tp + tn + fp + fn)
recall = tp/(tp+fn)
precision = tp/(tp+fp)
f_1 = (2*precision*recall)/(precision+recall)
print(f"accuracy: {accuracy}")
print(f"recall: {recall}")
print(f"precision: {precision}" )
print(f"f_1: {f_1}")

accuracy: 0.9077432255395436
recall: 0.337295690936107
precision: 1.0
f_1: 0.5044444444444445


In [129]:
test_dict = defaultdict(lambda: {"tp": 0, "fp": 0, "tn": 0, "fn": 0})
wrong_pred_dict = defaultdict(lambda:[])
correct_pred_dict = defaultdict(lambda:[])
for i, pred in enumerate(prediction):
    id = test_id_list[i]
    label = english_dataframe.iloc[id]["NE-COARSE-LIT"]
    token = english_dataframe.iloc[id]["TOKEN"]
    if label == "O" and pred == 0:
        test_dict["O"]["tp"] += 1
    elif label == "O" and pred == 1:
        test_dict["O"]["fn"] += 1
    elif label != "O" and pred == 0:
        test_dict[label]["fn"] += 1
        test_dict["O"]["fp"] += 1
        wrong_pred_dict[label].append(token)
    elif label != "O" and pred == 1:
        test_dict[label]["tp"] += 1
        test_dict["O"]["tn"] += 1
        correct_pred_dict[label].append(token)

In [130]:
for label in wrong_pred_dict:
    print(label)
    print(wrong_pred_dict[label])
    print("------------------------------------")

B-prod
['anti', 'anti', 'GAZETTE', 'MAGNA', 'A', 'Winy', 'Obseiver', 'Fayetteville', 'Wilmington', 'the', 'Weld', 'Putnam', 'Norfolk', 'Guide', 'Christian']
------------------------------------
I-prod
['-', 'federalifm', '-', 'federalifm', 'Gaz', 'CHARTA', 'SPLENDID', 'EDITION', 'OF', 'THE', 'Declaration', 'of', 'Independence', 'aw', '.', 'Observer', '-', 'xertiscr', 'Journal', 'Alkkt', 'County', 'Republican', 'ald', '¬', 'azine', 'and', 'Guide', 'Science', 'Science']
------------------------------------
B-loc
['Point', 'Guadaloupe', 'Point', 'America', 'Bengal', 'Bra', 'Brabant', 'Breda', 'Genoa', 'Genoa', 'Milford', 'State', 'Philadelphia', 'VIENNA', 'Vienna', 'Br.iuniu', 'Braunao', 'England', 'Frtnce', 'London', 'Dublin', 'Wilmington', 'Malta', 'Biistol', 'Cape', 'loGibraltar', 'Cagliari', 'Port', 'L', 'United', 'Paris', 'Portugal', 'Spain', 'Portugal', 'Lisbon', 'United', 'Virginia', 'lit', 'Sciota', 'Christ', 'Alexandria', 'the', 'Piince', 'America', 'Europe', 'Ireland', 'Pomonkey

In [132]:
for key in test_dict:
    if key == "O":
        continue
    print(key)
    temp_tp = test_dict[key]['tp']
    temp_fn = test_dict[key]["fn"]
    print(temp_tp/(temp_tp+temp_fn))
    print("------------------------")

B-prod
0.21052631578947367
------------------------
I-prod
0.275
------------------------
B-loc
0.3316062176165803
------------------------
I-loc
0.38202247191011235
------------------------
B-org
0.23076923076923078
------------------------
I-org
0.24731182795698925
------------------------
B-time
0.2727272727272727
------------------------
I-time
0.20689655172413793
------------------------
B-pers
0.4175257731958763
------------------------
I-pers
0.35947712418300654
------------------------


In [137]:
total_itterations = 1
performance_dict = defaultdict(lambda:{"avg_prec":0, "avg_recall": 0, "avg_acc": 0, "avg_f1":0})
recall_dict =  defaultdict(lambda: defaultdict(lambda:0))

for prob in np.arange(0.1, 1, step=0.1):
    for itter in range(total_itterations):
        total_sentences = len(set(sentence_idx))
        split = prob
        rng = default_rng()
        training_sentence_ids = rng.choice(total_sentences, size=int(total_sentences*split), replace=False)
        test_sentence_ids = [i for i in range(total_sentences) if i not in training_sentence_ids]

        training_id_list = []
        test_id_list = []
        for i, item in enumerate(sentence_idx):
            if item in training_sentence_ids:
                training_id_list.append(i)
            else:
                test_id_list.append(i)

        training_feature_matrix = np.array([feature_matrix[i] for i in training_id_list])
        training_label_matrix = np.array([label_matrix[i] for i in training_id_list])
        test_feature_matrix = np.array([feature_matrix[i] for i in test_id_list])
        test_label_matrix = np.array([label_matrix[i] for i in test_id_list])
        test_classifier = dt.custom_decision_tree(training_feature_matrix, training_label_matrix, discrete_idx_list=discrete_idx_list, min_samples=35)
        test_classifier.build_tree()

        custom_tree = test_classifier.tree
        
        tp = 0
        fp = 0
        fn = 0
        tn = 0
        total = 0
        prediction = custom_tree.predict_matrix(test_feature_matrix)
        prev_label = 0
        temp_dict = defaultdict(lambda: {"tp": 0, "fp": 0, "tn": 0, "fn": 0})
        for i, pred in enumerate(prediction):
            correct_label = test_label_matrix[i]
            id = test_id_list[i]
            label_txt = english_dataframe.iloc[id]["NE-COARSE-LIT"]
            if correct_label == 0 and pred == 0:
                tn += 1
            elif correct_label == 0 and pred != 0:
                fn += 1
            elif correct_label == 1 and pred == 0:
                fn += 1
            else:
                tp += 1
            
            if label_txt == "O" and pred == 0:
                temp_dict["O"]["tp"] += 1
            elif label_txt == "O" and pred == 1:
                temp_dict["O"]["fn"] += 1
            elif label_txt != "O" and pred == 0:
                temp_dict[label_txt]["fn"] += 1
                temp_dict["O"]["fp"] += 1
            elif label_txt != "O" and pred == 1:
                temp_dict[label_txt]["tp"] += 1
                temp_dict["O"]["tn"] += 1
        
        if tp == 0:
            precision = 0
            recall = 0
        else:
            precision = tp/(tp+fp)
            recall = tp/(tp+fn)
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f_1 = (2*precision*recall)/(precision+recall)
        performance_dict[prob]["avg_prec"] += precision/total_itterations
        performance_dict[prob]["avg_recall"] += recall/total_itterations
        performance_dict[prob]["avg_acc"] += accuracy/total_itterations
        performance_dict[prob]["avg_f1"] += f_1/total_itterations

        for key in temp_dict:
            tp = temp_dict[key]["tp"]
            fn = temp_dict[key]["fn"]
            if tp == 0:
                recall = 0
            else:
                recall = tp/(tp+fn)
            recall_dict[prob][key] += recall / total_itterations 

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
0.1
0.2
0.3


KeyboardInterrupt: 

In [138]:
for key in performance_dict:
    print(performance_dict[key])
    print("----------------")

{'avg_prec': 1.0, 'avg_recall': 0.2821755519655358, 'avg_acc': 0.897232287410377, 'avg_f1': 0.4401511969760605}
----------------


In [139]:
for key in recall_dict:
    print(recall_dict[key])
    print("----------------")

defaultdict(<function <lambda>.<locals>.<lambda> at 0x000002421B6BADC8>, {'O': 0.9128054891681443})
----------------
